In [8]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True

### Shakespeare Dataset

In [10]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt sonnets.txt

--2020-07-01 11:41:57--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 64.233.167.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘sonnets.txt.1’

sonnets.txt.1       100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-07-01 11:41:57 (73.6 MB/s) - ‘sonnets.txt.1’ saved [93578/93578]

--2020-07-01 11:41:57--  http://sonnets.txt/
Resolving sonnets.txt (sonnets.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘sonnets.txt’
FINISHED --2020-07-01 11:41:57--
Total wall clock time: 0.03s
Downloaded: 1 files, 91K in 0.001s (73.6 MB/s)


### Data Pre-processing

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
data = open('sonnets.txt').read()
corpus = data.lower().split('\n')
corpus

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 'when forty winters shall beseige thy brow,',
 "and dig deep trenches in thy beauty's field,",
 "thy youth's proud livery, so gazed on now,",
 "will be a tatter'd weed, of small worth held:",
 "then being ask'd where all thy beauty lies,",
 'where all the treasure of thy lusty days,',
 'to say, within thine own deep-sunken eyes,',

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'and': 1, 'the': 2, 'to': 3, 'of': 4, 'my': 5, 'i': 6, 'in': 7, 'that': 8, 'thy': 9, 'thou': 10, 'with': 11, 'for': 12, 'is': 13, 'love': 14, 'not': 15, 'but': 16, 'a': 17, 'me': 18, 'thee': 19, 'so': 20, 'be': 21, 'as': 22, 'all': 23, 'you': 24, 'his': 25, 'which': 26, 'when': 27, 'it': 28, 'this': 29, 'by': 30, 'your': 31, 'doth': 32, 'do': 33, 'from': 34, 'on': 35, 'or': 36, 'no': 37, 'then': 38, 'have': 39, 'what': 40, 'are': 41, 'if': 42, 'more': 43, 'mine': 44, 'their': 45, 'shall': 46, 'sweet': 47, 'time': 48, 'will': 49, 'they': 50, 'beauty': 51, 'nor': 52, 'eyes': 53, 'art': 54, 'her': 55, 'heart': 56, 'yet': 57, 'o': 58, 'than': 59, 'can': 60, 'should': 61, 'thine': 62, 'now': 63, 'where': 64, 'make': 65, 'one': 66, 'hath': 67, 'he': 68, 'fair': 69, 'still': 70, 'how': 71, 'eye': 72, 'him': 73, 'like': 74, 'true': 75, 'see': 76, 'am': 77, 'she': 78, 'those': 79, 'though': 80, 'being': 81, 'some': 82, 'every': 83, 'such': 84, 'own': 85, 'were': 86, 'dost': 87, 'who': 88, 'liv

In [15]:
tokenizer.texts_to_sequences(['without all ornament, itself and true,'])[0]

[266, 23, 419, 332, 1, 75]

In [17]:
input_sequences = []
tk = tokenizer.texts_to_sequences(['without all ornament, itself and true,'])[0]
for i in range(1,len(tk)):
  n_gram_sequence = tk[:i+1]
  input_sequences.append(n_gram_sequence)
print(input_sequences)

[[266, 23], [266, 23, 419], [266, 23, 419, 332], [266, 23, 419, 332, 1], [266, 23, 419, 332, 1, 75]]


In [18]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)
print(len(input_sequences))

15462


In [19]:
max_sequences_len = max([len(seq) for seq in input_sequences])
print(max_sequences_len)
input_sequences = np.array(pad_sequences(input_sequences, padding='pre', maxlen=max_sequences_len))

11


In [20]:
input_sequences[31]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 1372,    9],
      dtype=int32)

In [21]:
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [22]:
ys.shape

(15462, 3211)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, SimpleRNN, Embedding

In [26]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=256, input_length=max_sequences_len-1))
model.add(Bidirectional(LSTM(units=512, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 256)           822016    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 1024)          3149824   
_________________________________________________________________
dropout (Dropout)            (None, 10, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               1180672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 3211)              825227    
Total params: 5,977,739
Trainable params: 5,977,739
Non-trainable params: 0
____________________________________________

In [27]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(xs, ys, epochs=125, batch_size=128)

Epoch 1/125
121/121 [==============================] - 6s 48ms/step - loss: 0.0027 - accuracy: 0.0211
Epoch 2/125
121/121 [==============================] - 6s 47ms/step - loss: 0.0024 - accuracy: 0.0193
Epoch 3/125
121/121 [==============================] - 6s 46ms/step - loss: 0.0024 - accuracy: 0.0193
Epoch 4/125
121/121 [==============================] - 6s 47ms/step - loss: 0.0024 - accuracy: 0.0237
Epoch 5/125
121/121 [==============================] - 6s 47ms/step - loss: 0.0024 - accuracy: 0.0219
Epoch 6/125
121/121 [==============================] - 6s 46ms/step - loss: 0.0024 - accuracy: 0.0243
Epoch 7/125
121/121 [==============================] - 6s 46ms/step - loss: 0.0023 - accuracy: 0.0261
Epoch 8/125
121/121 [==============================] - 6s 46ms/step - loss: 0.0023 - accuracy: 0.0268
Epoch 9/125
121/121 [==============================] - 6s 46ms/step - loss: 0.0023 - accuracy: 0.0293
Epoch 10/125
121/121 [==============================] - 6s 46ms/step - loss: 0.002

In [28]:
model.save('shakespeare_model.h5')

In [29]:
from tensorflow.keras.models import load_model

In [30]:
model_final = load_model('shakespeare_model.h5')

#### Predict next word

In [37]:
def predict_next_words(seed_text, num_words_to_generate):
  for _ in range(num_words_to_generate):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequences_len-1, padding='pre')
    predicted = model_final.predict_classes(token_list, verbose=0)
    output_word=""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  print(seed_text)
  return seed_text

In [38]:
seed_text = "from art comes thou"
next_words = 100
generated_text = predict_next_words(seed_text, next_words)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
from art comes thou be the muses filed mad fitted smells 'will weep can their end ' rolling growth erred erred woe of nought worse lines me see well ' thee told hid tell thee thee to me free thought thee bright see told bright thee thou art true write me to me thought bright bright thee thee thou art free thought thee not bright so thee erred lack erred woe of you ' thee thee erred doth sort spend rolling free sheds sheds sheds bright so rage see not thee thou sorrow cross fall to thee you thee ' thee well ' thee


In [39]:
seed_text = "at the behest of the benevolent king"
next_words = 9
generated_text = predict_next_words(seed_text, next_words)

at the behest of the benevolent king so gentlest rose days erred level live arising cold


In [40]:
seed_text = "and the emperor said"
next_words = 9
generated_text = predict_next_words(seed_text, next_words)

and the emperor said i condemned which is in this date chest pace
